## Daniel Alonso, UID: 100444499

<img src='./images/1.png'>

<img src='./images/2.png'>

## Problem 1-a

Let $i$ be the source of a movement and $j$ be the destination

Let $x$ be movements from one node to another, where $x_{i,j}$ represents the movement from node $i$ to node $j$

Integer optimization formulation:

minimize $44x_{1,2} + 18x_{1,3} + 85x_{1,4} + 17x_{2,5} + 13x_{2,6} + 55x_{3,5} + 53x_{3,6} + 7x_{3,7} + 26x_{4,6} + 66x_{4,7} + 6x_{5,8} + 7x_{5,9} + 26x_{6,8} + 66x_{6,9} + 26x_{6,10} + 6x_{7,9} + 60x_{7,10} + 58x_{8,11} + 70x_{9,11} + 27x_{10,11}$

subject to:

$x_{1,2} + x_{1,3} + x_{1,4} = 1$

$x_{1,2} - x_{2,5} - x_{2,6} = 0 $

$x_{1,3} - x_{3,5} - x_{3,6} - x_{3,7} = 0$

$x_{1,4} - x_{4,6} - x_{4,7} = 0$

$x_{2,5} + x_{3,5} - x_{5,8} - x_{5,9} = 0$

$x_{2,6} + x_{3,6} + x_{4,6} - x_{6,8} - x_{6,9} - x_{6,10} = 0$

$x_{3,7} + x_{4,7} - x_{7,9} - x_{7,10} = 0$

$x_{5,8} + x_{6,8} - x_{8,11} = 0$

$x_{5,9} + x_{6,9} + x_{7,9} - x_{9,11} = 0$

$x_{6,10} + x_{7,10} - x_{10,11} = 0$

$- x_{8,11} - x_{9,11} - x_{10,11} = -1$

The integer optimization formulation (in my opinion) is more appropriate. It not only will provide the optimal integer solution but it also describes the problem better and more accurately.

## Problem 1-b

In [18]:
# Gurobi-python implementation
from gurobipy import *

In [22]:
nodes, supply = multidict({
         1: 1,
         2: 0,
         3: 0,
         4: 0,
         5: 0,
         6: 0,
         7: 0,
         8: 0,
         9: 0,
         10: 0,
         11: -1})
 
arcs, distance = multidict({
    (1, 2): 44,
    (1, 3): 18,
    (1, 4): 85,
    (2, 5): 17,
    (2, 6): 13,
    (3, 5): 55,
    (3, 6): 53,
    (3, 7): 7,
    (4, 6): 26,
    (4, 7): 66,
    (5, 8): 6,
    (5, 9): 7,
    (6, 8): 26,
    (6, 9): 66,
    (6, 10): 26,
    (7, 9): 6,
    (7, 10): 60,
    (8, 11): 58,
    (9, 11): 70,
    (10, 11): 27})

num_nodes = 11

# Optimization model
m = Model('shortest_path')
x = m.addVars(arcs, obj=distance, name="dist")
m.addConstrs((x.sum(i, '*') - x.sum('*', i) == supply[i] for i in nodes), "supply")
m.ModelSense = GRB.MINIMIZE
m.optimize()


Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 11 rows, 20 columns and 40 nonzeros
Model fingerprint: 0xd11ad6da
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 6 columns
Presolve time: 0.02s
Presolved: 5 rows, 14 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5000000e+01   2.000000e+00   0.000000e+00      0s
       2    1.0100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds
Optimal objective  1.010000000e+02


In [29]:
# Printing solution
# Variable information including sensitivity information
varnames = [f"x[{tuple(x)}]" for x in tuple(arcs)]
for var, v in zip(varnames,m.getVars()):
    print(f"{var} = {v.X}")

x[(1, 2)] = 0.0
x[(1, 3)] = 1.0
x[(1, 4)] = 0.0
x[(2, 5)] = 0.0
x[(2, 6)] = 0.0
x[(3, 5)] = 0.0
x[(3, 6)] = 0.0
x[(3, 7)] = 1.0
x[(4, 6)] = 0.0
x[(4, 7)] = 0.0
x[(5, 8)] = 0.0
x[(5, 9)] = 0.0
x[(6, 8)] = 0.0
x[(6, 9)] = 0.0
x[(6, 10)] = 0.0
x[(7, 9)] = 1.0
x[(7, 10)] = 0.0
x[(8, 11)] = 0.0
x[(9, 11)] = 1.0
x[(10, 11)] = 0.0


We can clearly see the shortest route from node 1 to node 11 is the following:

$x_{1,3} \rightarrow x_{3,7} \rightarrow x_{7,9} \rightarrow x_{9,11}$

<img src='./images/path.png'>

## Problem 1-c

In [36]:
# Optimal shadow prices
for n,c in enumerate(m.getConstrs()):
    print(f"{c.ConstrName} : shadow price = {int(c.Pi)}")

supply[1] : shadow price = 48
supply[2] : shadow price = 4
supply[3] : shadow price = 30
supply[4] : shadow price = -37
supply[5] : shadow price = 0
supply[6] : shadow price = 0
supply[7] : shadow price = 23
supply[8] : shadow price = 5
supply[9] : shadow price = 17
supply[10] : shadow price = -26
supply[11] : shadow price = -53


In [37]:
# Printing solution
# Variable information including sensitivity information
varnames = [f"x[{tuple(x)}]" for x in tuple(arcs)]
for var, v in zip(varnames,m.getVars()):
    print(f"{var} = {int(v.X)}, reduced cost = {abs(int(v.RC)):.2f},")

x[(1, 2)] = 0, reduced cost = 0.00,
x[(1, 3)] = 1, reduced cost = 0.00,
x[(1, 4)] = 0, reduced cost = 0.00,
x[(2, 5)] = 0, reduced cost = 13.00,
x[(2, 6)] = 0, reduced cost = 9.00,
x[(3, 5)] = 0, reduced cost = 25.00,
x[(3, 6)] = 0, reduced cost = 23.00,
x[(3, 7)] = 1, reduced cost = 0.00,
x[(4, 6)] = 0, reduced cost = 63.00,
x[(4, 7)] = 0, reduced cost = 126.00,
x[(5, 8)] = 0, reduced cost = 11.00,
x[(5, 9)] = 0, reduced cost = 24.00,
x[(6, 8)] = 0, reduced cost = 31.00,
x[(6, 9)] = 0, reduced cost = 83.00,
x[(6, 10)] = 0, reduced cost = 0.00,
x[(7, 9)] = 1, reduced cost = 0.00,
x[(7, 10)] = 0, reduced cost = 11.00,
x[(8, 11)] = 0, reduced cost = 0.00,
x[(9, 11)] = 1, reduced cost = 0.00,
x[(10, 11)] = 0, reduced cost = 0.00,


<img src='./images/3.png'>

## Problem 2-a

## Problem 2-b

In [16]:
years = range(0,3)
projects = range(0,4)

In [ ]:
# primal objective coefficients
r_coeff = [44, 18, 85, 17, 13, 55, 53, 7, 26, 66, 6, 7, 26, 66, 26, 6, 60, 58, 70, 27] 

# left-hand side (LHS) coefficients (matrix A)
A_coeff = [[1, 1, 1],
           [1, -1, -1],
           [1, -1, -1, -1],
           [1, -1, -1],
           [1, 1, -1, -1],
           [1, 1, 1, -1, -1, -1],
           [1, 1, -1, -1],
           [1, 1, -1],
           [1, 1, 1, -1],
           [1, 1, -1],
           [1, 1, 1]]

# right-hand side (RHS) coefficients
b_coeff = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

r = {j : r_coeff[j] for j in projects}

A = {i : {j : A_coeff[i][j] for j in projects} 
    for i in years}

b = {i : b_coeff[i] for i in years}

model = Model('projectsip1')

x = model.addVars(projects, name="x", vtype=GRB.BINARY)

# Capacity constraints
model.addConstrs((quicksum(A[i][j] * x[j] for j in projects)
                           <= b[i] 
                            for i in years))

# Variable upper bound constraints
model.addConstrs((x[j] <= 1 for j in projects))

# Objective
obj = quicksum(r[j] * x[j] for j in projects)

model.setObjective(obj, GRB.MINIMIZE)

# disable Presolve
model.setParam(GRB.Param.Presolve, 0)
# disable Heuristics
model.setParam(GRB.Param.Heuristics, 0)
# disable Cuts
model.setParam(GRB.Param.Cuts, 0)
        
model.optimize()

# Display solution (print the name of each variable and the solution value)
print('--------------------------------')
print('\nOptimal solution:\n')

print('Variable Information:')
                 
for v in model.getVars():
    print("%s %s %8.2f" % 
              (v.Varname, "=", v.X))
    
    print(" ")
        
print('\nOptimal objective value: %g' % model.objVal)


<img src='./images/4.png'>

## Problem 3-a

## Problem 3-b